In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pprint
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

import os

from salaries_functions import write_jobs_to_txt, scrape_pages

In [ ]:
netherlands_url = 'https://techpays.com/europe/netherlands'
response = requests.get(netherlands_url)

if not os.path.exists('netherlands_salaries.txt'):
    with open('netherlands_salaries.txt', 'w', encoding= 'utf-8') as f:
        pass

In [ ]:
# driver = webdriver.Chrome()
# driver.get(country_url)
# time.sleep(3)

# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')

# driver.quit()

# with open('response_content.txt', 'w', encoding= 'utf-8') as f:
#     f.write(soup.prettify())

In [ ]:
scrape_pages(netherlands_salaries)

Wrote jobs dictionary.
Clicked next button for the 1th time
Wrote jobs dictionary.
Clicked next button for the 2th time
Wrote jobs dictionary.
Clicked next button for the 3th time
Wrote jobs dictionary.
Clicked next button for the 4th time
Wrote jobs dictionary.
Clicked next button for the 5th time
Wrote jobs dictionary.
Clicked next button for the 6th time
Wrote jobs dictionary.
Clicked next button for the 7th time
Wrote jobs dictionary.
Clicked next button for the 8th time
Wrote jobs dictionary.
Clicked next button for the 9th time
Wrote jobs dictionary.
Clicked next button for the 10th time
Wrote jobs dictionary.
Clicked next button for the 11th time
Wrote jobs dictionary.
Clicked next button for the 12th time
Wrote jobs dictionary.
Clicked next button for the 13th time
Wrote jobs dictionary.
Clicked next button for the 14th time
Wrote jobs dictionary.
Clicked next button for the 15th time
Wrote jobs dictionary.
Clicked next button for the 16th time
Wrote jobs dictionary.
Clicked ne

KeyboardInterrupt: 